In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(usecwd=True), override=True)

import os
print("OPENAI ok:", bool(os.getenv("OPENAI_API_KEY")))
print("REPLICATE ok:", bool(os.getenv("REPLICATE_API_TOKEN")))

OPENAI ok: True
REPLICATE ok: True


In [3]:
# 기본 임포트
from typing import List, Dict, Literal, TypedDict
from pydantic import BaseModel, Field
from typing_extensions import Annotated

from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

In [4]:
# 1) 스키마 정의 (EmotionResult / MusicBrief / GraphState)
Valence = Annotated[float, Field(ge=-1.0, le=1.0)]
Arousal = Annotated[float, Field(ge=0.0, le=1.0)]
BPM     = Annotated[int,   Field(ge=50,  le=140)]
DurSec  = Annotated[int,   Field(ge=60,   le=90)] # 음악 재생시간(60초 ~ 90초)

class EmotionResult(BaseModel):
    primary: str
    valence: Valence = 0.0
    arousal: Arousal = 0.5
    confidence: Arousal = 0.7
    reasons: str = "—"

class MusicBrief(BaseModel):
    mood: str
    bpm: BPM = 90
    key: str = "C major"
    duration_sec: DurSec = 60               # 음악 기본 재생시간 60초
    instruments: List[str] = []
    style_tags: List[str] = []
    prompt: str  # 영어 프롬프트

class GraphState(TypedDict, total=False):
    user_text: str
    emotion: EmotionResult
    brief: MusicBrief
    audio_path: str
    provider_used: Literal["replicate"]
    meta: Dict


In [5]:
# 2) LLM 헬퍼 (OpenAI)
def get_llm():
    # 스키마 맞춤 출력이 중요 → 낮은 temperature
    return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)


In [6]:
# 3) 감정 분석 노드
def analyze_emotion_node(state: GraphState) -> GraphState:
    llm = get_llm()
    sys = (
        "당신은 심리 정서를 요약하는 분석가입니다. "
        "사용자 텍스트에서 주요 감정을 한 단어(또는 짧은 구)로 도출하고, "
        "valence(-1~1), arousal(0~1), confidence(0~1)을 추정하세요. "
        "반드시 EmotionResult(JSON 스키마)에 맞춰 응답하세요."
    )
    structured = llm.with_structured_output(EmotionResult)
    result = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":state["user_text"]}
    ])
    state["emotion"] = result
    return state

In [ ]:
# 4) 음악 브리프 노드
def compose_brief_node(state: GraphState) -> GraphState:
    llm = get_llm()
    emo: EmotionResult = state["emotion"]
    sys = (
        "너는 음악 감독이다. 아래 감정 분석과 사용자 텍스트를 참고해 "
        "치유/안정 목적의 짧은 BGM을 위한 Music Brief를 JSON으로 만들어라. "
        "bpm=50~140, duration_sec=60~90, key는 'C major' 같은 형식. "
        "prompt는 영어로 핵심 악기/무드/다이내믹을 간결히 포함."
    )
    usr = (
        f"# Emotion\nprimary={emo.primary}, valence={emo.valence}, "
        f"arousal={emo.arousal}, confidence={emo.confidence}\n\n"
        f"# Text\n{state['user_text']}\n"
    )
    structured = llm.with_structured_output(MusicBrief)
    brief = structured.invoke([
        {"role":"system","content":sys},
        {"role":"user","content":usr}
    ])

# ▼ duration 보정 (60~90초로 강제)
    if brief.duration_sec < 60:
        brief = brief.model_copy(update={"duration_sec": 60})
    elif brief.duration_sec > 90:
        brief = brief.model_copy(update={"duration_sec": 90})

    state["brief"] = brief
    return state